Getting started guide https://www.tensorflow.org/get_started/get_started

In [1]:
import tensorflow as tf

In [2]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) #float32 implicitly
print (node1, node2)

(<tf.Tensor 'Const:0' shape=() dtype=float32>, <tf.Tensor 'Const_1:0' shape=() dtype=float32>)


the above declared nodes for a computational graph. That's why the output does not include the actual value.  The graph must be run in a 'session' to produce the results

In [4]:
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [5]:
node3 = tf.add(node1, node2)
print ('node3: ', node3)
print('sess.run(node3): ', sess.run(node3))

('node3: ', <tf.Tensor 'Add:0' shape=() dtype=float32>)
('sess.run(node3): ', 7.0)


above only uses constant values. TF allows for nodes to be parametrized to accept external inputs known as PLACE HOLDERS.  placeholder is a promise to provide a value later

In [6]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a+b #or tf.add(a,b)

print(sess.run(adder_node, {a:3, b:4.5}))
print(sess.run(adder_node, {a:[1,3], b:[2,4]}))

7.5
[ 3.  7.]


In [8]:
add_and_triple = adder_node*3
print(sess.run(add_and_triple, {a:3, b:4.5}))

22.5


    for machine learning, we need to make the model trainable, to get new outputs with the same input. VARIABLES allows us to add trainable parameeters to a graph

In [10]:
W = tf.Variable([.3])
b = tf.Variable([-.3])
x = tf.placeholder(tf.float32)
lin_model = W * x + b

    Variables are not initialized at the tf.Variable call (unlike tf.Constant call).  To init all the variables in a TF program, need to call the below operation

In [11]:
init = tf.global_variables_initializer()
sess.run(init)

In [13]:
print(sess.run(lin_model, {x:[1,2,3,4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


    add a loss function and desired output 'y' to measure accuracy

In [17]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(lin_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

23.66


change W and b to the ideal values. Assign is used to assign values to variables

In [19]:
fixW = tf.assign(W, [-1])
fixb = tf.assign(b, [1])
sess.run([fixW, fixb])
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

0.0


using optimizer (GD) to minimize loss


In [20]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

sess.run(init) #reset values to incorrect defaults
for i in range(1000):
    sess.run(train, {x:[1,2,3,4], y:[0,-1,-2,-3]})
print sess.run([W,b])

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


## tf.contrib.learn, a high level TF API

Same linear regression as above but using tf.contrib.learn

In [25]:
import numpy as np

#declare list of features
features = [tf.contrib.layers.real_valued_column("x", dimension=1)]

#estimator
estimator = tf.contrib.learn.LinearRegressor(feature_columns=features)

x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x":x_train}, y_train,
                                             batch_size=4,
                                             num_epochs=1000)

eval_input_fn = tf.contrib.learn.io.numpy_input_fn(
   {"x":x_eval}, y_eval, batch_size=4, num_epochs=1000)

estimator.fit(input_fn=input_fn, steps=1000)

train_loss = estimator.evaluate(input_fn=input_fn)
eval_loss = estimator.evaluate(input_fn=eval_input_fn)

print("train loss: %r"% train_loss)
print("eval loss: %r"% eval_loss)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11affb990>, '_model_dir': '/var/folders/jm/xf73ff317kq6hr6_qnsm6zqm6lnrwd/T/tmpLkEf7Y', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_evaluation_master': '', '_master': ''}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no lon

In [22]:
features

[_RealValuedColumn(column_name='x', dimension=1, default_value=None, dtype=tf.float32, normalizer=None)]